In [139]:
import os
import copy
import warnings

import torch
import random
import pandas as pd
import numpy  as np
import torch.nn as nn
from src.processing import *
from torchvision import models
from torch.utils.data import DataLoader
import src.model_fitting as model_fitting

warnings.filterwarnings("ignore")

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.deterministic = True

In [ ]:
!unzip private_dataset.zip

In [133]:
#images_amount = check_images_amount()

annotations = pd.read_csv("annotation_file.csv")
annot_test = pd.read_csv("annotation_file_test.csv").sort_values(by="id")

NEW_SIZE, BATCH_SIZE = 224, 256
full_train = CarDataset(annotation_file=annotations, img_dir="train", new_size=NEW_SIZE, mode="train")
full_train_dataloader = DataLoader(full_train, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = CarDataset(annotation_file=annot_test, img_dir="public_test", new_size=NEW_SIZE, mode="test")
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [136]:
#!g1.1
model = models.resnet34(pretrained=True)
model.fc = nn.Linear(512, 11)
model = model.cuda()

torch.cuda.empty_cache()

In [137]:
#!g1.1
fitting_info = model_fitting.train(full_train_dataloader, None, model=model, epochs=20, validate=False)

torch.save(copy.deepcopy(model.state_dict()), 'resnet18_fitting_weights.pth')
torch.cuda.empty_cache()

[0]: tr_loss: 0.27203 tr_f1: 0.91451 
[1]: tr_loss: 0.11316 tr_f1: 0.96402 
[2]: tr_loss: 0.08 tr_f1: 0.97453 
[3]: tr_loss: 0.03651 tr_f1: 0.9886 
[4]: tr_loss: 0.02224 tr_f1: 0.99319 
[5]: tr_loss: 0.01433 tr_f1: 0.9957 
[6]: tr_loss: 0.00719 tr_f1: 0.99797 
[7]: tr_loss: 0.00364 tr_f1: 0.99908 
[8]: tr_loss: 0.00297 tr_f1: 0.99933 
[9]: tr_loss: 0.00181 tr_f1: 0.9995 
[10]: tr_loss: 0.00142 tr_f1: 0.99957 
[11]: tr_loss: 0.00126 tr_f1: 0.99963 
[12]: tr_loss: 0.00076 tr_f1: 0.99988 
[13]: tr_loss: 0.00081 tr_f1: 0.99973 
[14]: tr_loss: 0.00068 tr_f1: 0.99983 
[15]: tr_loss: 0.0005 tr_f1: 0.9999 
[16]: tr_loss: 0.00062 tr_f1: 0.9999 
[17]: tr_loss: 0.0005 tr_f1: 0.9999 
[18]: tr_loss: 0.00057 tr_f1: 0.99988 
[19]: tr_loss: 0.00048 tr_f1: 0.99983 


epoch: 100%|██████████| 20/20 [1:27:16<00:00, 261.84s/it]


In [138]:
#!g1.1
inverse_target_mapping = {path_idx: path for path_idx, path in enumerate(os.listdir("train"))} 

preds = model_fitting.predict(model, test_dataloader, inverse_target_mapping=inverse_target_mapping)
if((len(preds) == images_amount["test"]) and (len(np.unique(preds)) == 11)):
    make_submit("submit8_aug", preds)

=====SUBMITION DONE=====


## private submit

---

In [167]:
import src.make_annotation_file as annot

In [169]:
annot.make()

Plese type mode(train or test)


> test


Please type annotation file name


> annotation_file_private


Please type basic directory


> private_test


=====ANNOTATION FILE CREATED=====


In [170]:
annot_test = pd.read_csv("annotation_file_private.csv").sort_values(by="id")
NEW_SIZE, BATCH_SIZE = 224, 256
test_dataset = CarDataset(annotation_file=annot_test, img_dir="private_test", new_size=NEW_SIZE, mode="test")
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [171]:
#!g1.1
model = models.resnet34(pretrained=True)
model.fc = nn.Linear(512, 11)
model = model.cuda()
model.load_state_dict(torch.load("./resnet34_fitting_weights.pth"))

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /tmp/xdg_cache/torch/hub/checkpoints/resnet34-b627a593.pth


<All keys matched successfully>

In [172]:
#!g1.1
inverse_target_mapping = {path_idx: path for path_idx, path in enumerate(os.listdir("train"))} 

preds = model_fitting.predict(model, test_dataloader, inverse_target_mapping=inverse_target_mapping)
if((len(preds) == len(os.listdir("./private_test"))) and (len(np.unique(preds)) == 11)):
    make_submit("private_submit", preds)

=====SUBMITION DONE=====
